In [ ]:
!pip install langchain_google_vertexai

In [ ]:
!pip install langchain_community

In [ ]:
!pip install neo4j

In [ ]:
!pip install tiktoken

In [ ]:
!pip install json-repair

In [ ]:
!nvidia-smi

In [ ]:
!pip install ipython-cypher


In [ ]:
!pip install -U langchain-ollama



In [ ]:
!pip install langchain-google-genai

In [ ]:
%load_ext cypher

## Neo4j Advanced rag 
https://github.com/langchain-ai/langchain/tree/master/templates/neo4j-advanced-rag this is has open ai reference and below you can find the gcp model implementation

Future work

Comparitive study with https://python.langchain.com/v0.1/docs/use_cases/graph/constructing/ approach

![Architecture](./Architecture.png)

In [1]:
from pathlib import Path
from typing import List, Optional
from langchain_google_vertexai import (
    VertexAI,
    ChatVertexAI,
    VertexAIEmbeddings,
    VectorSearchVectorStore,
)
from google.cloud import aiplatform
from langchain_community.vectorstores import Neo4jVector

from langchain_community.document_loaders import TextLoader
from langchain_community.graphs import Neo4jGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_text_splitters import TokenTextSplitter
from neo4j.exceptions import ClientError
import os
from vertexai.generative_models import GenerativeModel, Image
from vertexai.language_models import TextEmbeddingModel, TextEmbeddingInput
from langchain_google_vertexai import ChatVertexAI, HarmBlockThreshold, HarmCategory
from unstructured.partition.pdf import partition_pdf
from langchain_text_splitters import CharacterTextSplitter
from typing import List, Tuple
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField, RunnableParallel,RunnableLambda
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
import base64
from langchain_core.messages import AIMessage, HumanMessage
import typing
from langchain_experimental.graph_transformers import LLMGraphTransformer

from langchain_community.chat_models import ChatOllama


In [2]:
url = "bolt://10.113.8.4:8085"
username ="neo4j"
password = "password"
os.environ["NEO4J_URI"] = url
os.environ["NEO4J_USERNAME"] = username
os.environ["NEO4J_PASSWORD"] = password

In [3]:
PROJECT_ID = "edl-idaas-rnd-platform-d5ae"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
aiplatform.init(project=PROJECT_ID, location=LOCATION)


In [4]:
# MATCH (n) MATCH ()-[r]->() RETURN n, r

In [5]:
# MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n, r;

In [6]:
pdf_folder_path = "/home/sridhanya_ganapathi_team_neustar/rag/data/images"
pdf_file_name = "/home/sridhanya_ganapathi_team_neustar/rag/data/contract_agreement.pdf"
Pdf_title="contract_agreement.pdf"

In [7]:
image_pdf_folder_path = "/home/sridhanya_ganapathi_team_neustar/rag/data/images"
pdf_file_name = "/home/sridhanya_ganapathi_team_neustar/rag/data/contract_agreement.pdf"

# Extract images, tables, and chunk text from a PDF file.
raw_pdf_elements = partition_pdf(
    filename=pdf_file_name,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=pdf_folder_path,
    extract_image_block_types=["Image", "Table"],
    extract_image_block_output_dir=image_pdf_folder_path,
    extract_image_block_to_payload=False
)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:


# Categorize extracted elements from a PDF into tables and texts.
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))

In [9]:
len(texts)

5

In [10]:
texts[0]

'Ni iE P.O. Box 1749 Halifax, Nova Scotia REGIONAL MUNICIPALITY B3J 3A5 Canada\n\nHalifax Regional Council August 6, 2013\n\nItem No. 14.1\n\nTO:         Mayor Savage and Members of Halifax Regional Council                  SUBMITTED BY:  ___________________________________________________________                                       Richard Butts, Chief Administrative Officer         __________________________________________________________  Mike Labrecque, Deputy Chief Administrative Officer      DATE:      July 31, 2013      SUBJECT:   Non-Disclosure Agreement with the Province of Nova Scotia   \n\n  \n\n     \n\nORIGIN\n\nOn June 25, 2013, Regional Council authorized the CAO to negotiate and execute an agreement for SAP support services with the Province of Nova Scotia.\n\nLEGISLATIVE AUTHORITY\n\nUnder section 58(5) of the HRM Charter, Regional Council has the power to make and carry out a contract. Under section 10(3) of the HRM Charter, the Mayor and Clerk may sign any documen

In [11]:
# Optional: Enforce a specific token size for texts
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=5000, chunk_overlap=0
)

In [12]:
# Embeddings & LLM models
# embeddings = OpenAIEmbeddings()
embedding_dimension = 768
# llm = ChatOpenAI(temperature=0)

# Loading Gemini Pro Vision Model
multimodal_model = GenerativeModel("gemini-1.5-pro-001")
# Initializing embedding model
EMBEDDING_MODEL =VertexAIEmbeddings(model_name="textembedding-gecko@003", dimension=768) #TextEmbeddingModel.from_pretrained("textembedding-gecko@003")

# Loading Gemini Pro Model
llm = ChatVertexAI(
    model_name="gemini-1.5-pro-001",
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },
)

ollamallm = ChatOllama(base_url='http://10.113.8.4:8086',
    model="llama3:latest", temperature=0)



In [13]:
# llm_transformer = LLMGraphTransformer(llm=llm,
#                                       allowed_nodes=["Person", "Country", "Organization", "Location", "Legal Authority","Effective Date", "Signed Date"],
#                                       allowed_relationships=["signed by", "Signed on", "Section", "Report Prepared by", 'Report Signed by', 'Originally signed', 'Submitted by'],
#                                       strict_mode=False)

In [14]:
# llm_transformer = LLMGraphTransformer(llm=llm,     allowed_nodes=["Person", "Country", "Organization"],
#     allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],strict_mode=False)

In [15]:
llm_transformer = LLMGraphTransformer(llm=ollamallm, allowed_nodes=["Person", "Country", "Organization", "Location", "Effective Date", "Signed Date", "heading"],
                                      allowed_relationships=["signed by", "Signed on", "Section"],
                                  strict_mode=False)

In [16]:
doc1 = text_splitter.create_documents(texts)

In [17]:

try:
    graph1 = llm_transformer.convert_to_graph_documents(doc1)
except Exception as e:
    print(f"Error processing document: {e}") 
# # Output the graph for inspection

# print(graph1)

 

In [18]:
len(graph1)

5

In [19]:
print(f"Nodes:{graph1[0].nodes}")
print(f"Relationships:{graph1[1].relationships}")

Nodes:[Node(id='Richard Butts, Chief Administrative Officer', type='Person'), Node(id='Halifax Regional Council', type='Organization'), Node(id='Mayor Savage and Members of Halifax Regional Council', type='Organization'), Node(id='Mike Labrecque, Deputy Chief Administrative Officer', type='Person'), Node(id='Non-Disclosure Agreement with the Province of Nova Scotia', type='Agreement')]
Relationships:[Relationship(source=Node(id='David Greener', type='Person'), target=Node(id='Legal Services', type='Organization'), type='PREPARED_BY'), Relationship(source=Node(id='Martin Ward, Q.C.', type='Person'), target=Node(id='Legal Services', type='Organization'), type='ACTING_DIRECTOR'), Relationship(source=Node(id='Halifax Regional Municipality', type='Organization'), target=Node(id='Her Majesty the Queen in right of the Province of Nova Scotia', type='Government'), type='PARTY_TO')]


In [26]:
ngraph = Neo4jGraph()

In [21]:
len(graph1)

5

In [22]:
ngraph.add_graph_documents(graph1,
                    baseEntityLabel=True)



In [57]:
MODEL_NAME = "gemini-1.5-pro-001"


# Generate summaries of text elements
def generate_text_summaries(
    texts: List[str], tables: List[str], summarize_texts: bool = False
) -> Tuple[List, List]:
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a elaborate concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
    prompt = PromptTemplate.from_template(prompt_text)
    empty_response = RunnableLambda(
        lambda x: AIMessage(content="Error processing document")
    )
    # Text summary chain
    model = VertexAI(
        temperature=0, model_name=MODEL_NAME, max_output_tokens=1024
    ).with_fallbacks([empty_response])
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts:
        if summarize_texts:
            text_summaries = summarize_chain.batch(texts, {"max_concurrency": 1})
        else:
            text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 1})

    return text_summaries, table_summaries


# Get text, table summaries
text_summaries, table_summaries = generate_text_summaries(
    texts, tables, summarize_texts=True
)

In [58]:
def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    """Make image summary"""
    model = ChatVertexAI(model_name="gemini-pro-vision", max_output_tokens=1024)

    msg = model(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/png;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content


def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval.
    If it's a table, extract all elements of the table.
    If it's a graph, explain the findings in the graph.
    Do not include any numbers that are not mentioned in the image.
    """

    # Apply to images
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt))

    return img_base64_list, image_summaries


# Image summaries
img_base64_list, image_summaries = generate_img_summaries(image_pdf_folder_path)

In [59]:
combine_all_texts = text_summaries + table_summaries + image_summaries 
joined_texts = " ".join(combine_all_texts)
texts_4k_token = text_splitter.split_text(joined_texts)

In [60]:
len(texts_4k_token)

1

In [61]:
docs = text_splitter.create_documents(texts_4k_token)
# print(docs[0].page_content)

In [62]:
# Ingest Parent-Child node pairs #  :all  passed
parent_splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=24)
child_splitter = TokenTextSplitter(chunk_size=256, chunk_overlap=24)
parent_documents = parent_splitter.split_documents(docs)

parent_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
child_splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=24)
parent_documents = parent_splitter.split_documents(docs)

In [63]:
len(parent_documents)

2

In [64]:
embedding_dimension = 768

In [65]:
ngraph = Neo4jGraph()

In [66]:
for i, parent in enumerate(parent_documents):
    child_documents = child_splitter.split_documents([parent])
    print(len(parent.page_content))

5118
2012


In [67]:
for i, parent in enumerate(parent_documents):
    child_documents = child_splitter.split_documents([parent])
    print(len(parent.page_content))
    params = {
        "parent_text": parent.page_content,
        "parent_id": i,
        "parent_embedding": EMBEDDING_MODEL.embed_query(parent.page_content), #EMBEDDING_MODEL.get_embeddings([parent.page_content])[0].values,
        "children": [
            {
                "text": c.page_content,
                "id": f"{i}-{ic}",
                "embedding": EMBEDDING_MODEL.embed_query(c.page_content) #EMBEDDING_MODEL.get_embeddings([c.page_content])[0].values,
            }
            for ic, c in enumerate(child_documents)
        ],
    }
     # Ingest data
    ngraph.query(
        """
    MERGE (p:Parent {id: $parent_id})
    SET p.text = $parent_text
    WITH p
    CALL db.create.setVectorProperty(p, 'embedding', $parent_embedding)
    YIELD node
    WITH p 
    UNWIND $children AS child
    MERGE (c:Child {id: child.id})
    SET c.text = child.text
    MERGE (c)<-[:HAS_CHILD]-(p)
    WITH c, child
    CALL db.create.setVectorProperty(c, 'embedding', child.embedding)
    YIELD node
    RETURN count(*)
    """,
        params,
    )
    # Create vector index for child
    try:
        ngraph.query(
            "CALL db.index.vector.createNodeIndex('parent_document', "
            "'Child', 'embedding', 768, 'cosine')",
            {},
        )
    except ClientError:  # already exists
        pass
    # Create vector index for parents
    try:
        ngraph.query(
            "CALL db.index.vector.createNodeIndex('typical_rag', "
            "'Parent', 'embedding', 768, 'cosine')",
            {},
        )
    except ClientError:  # already exists
        pass
# CALL db.index.vector.createNodeIndex('parent_document', 'Child', 'embedding', 768, 'cosine')

5118
2012


![neo4j](./Summary.png)

![neo4j](./Parent.png)

![neo4j](./Child.png)

In [68]:
llm

ChatVertexAI(project='edl-idaas-rnd-platform-d5ae', model_name='gemini-1.5-pro-001', full_model_name='projects/edl-idaas-rnd-platform-d5ae/locations/us-central1/publishers/google/models/gemini-1.5-pro-001', client_options=ClientOptions: {'api_endpoint': 'us-central1-aiplatform.googleapis.com', 'client_cert_source': None, 'client_encrypted_cert_source': None, 'quota_project_id': None, 'credentials_file': None, 'scopes': None, 'api_key': None, 'api_audience': None, 'universe_domain': None}, default_metadata=(), model_family=<GoogleModelFamily.GEMINI_ADVANCED: '2'>, safety_settings={<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 1>: <HarmBlockThreshold.BLOCK_LOW_AND_ABOVE: 1>})

In [69]:
# Ingest frequently asked questions  questions


class Questions(BaseModel):
    """Generating hypothetical questions about text."""

    questions: List[str] = Field(
        ...,
        description=(
            "Generated hypothetical questions based on  the information from the text"
        ),
    )


In [70]:

questions_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            (
                "Use the given format to generate frequently asked questions from the "
                "following input: {input}"
            ),
        ),
    ]
)
question_chain = questions_prompt | llm.with_structured_output(Questions)


In [71]:
question_chain = questions_prompt | llm.with_structured_output(Questions)

In [72]:
len(parent_documents)

2

In [73]:
embedding_dimension

768

In [74]:
for i, parent in enumerate(parent_documents):
    resp = question_chain.invoke(parent.page_content)
    if (resp != None):
        questions = resp.questions
        print(questions)
        print('-------')

['What is the agreement about?', 'What are the key restrictions imposed by the agreement?', 'What happens to the confidential information after the agreement ends?', 'What are the legal implications of breaching the agreement?', 'Under what legal framework is this agreement established?']
-------
['What is the significance of the wavy line in the Halifax Regional Municipality logo?', 'What is the connection between the Halifax Regional Municipality logo and the content of the letter from the Chief Administrative Officer?', 'Why was a Non-Disclosure Agreement necessary between Halifax Regional Municipality and the Province of Nova Scotia in 2013?', 'What was the purpose of the meeting announced in the notice from David Greener and Martin Ward?', 'What is the process for the public to provide input during the public hearing on the proposed subdivision development?', 'What are the key proposed changes to the zoning regulations for the downtown area?', 'Who are David Greener and Martin War

In [75]:
for i, parent in enumerate(parent_documents):
    resp = question_chain.invoke([parent.page_content])
    if (resp != None):
        questions = resp.questions
        params = {
            "parent_id": i,
            "questions": [
                {"text": q, "id": f"{i}-{iq}", "embedding": EMBEDDING_MODEL.embed_query(q)} #EMBEDDING_MODEL.get_embeddings([q])[0].values}
                for iq, q in enumerate(questions)
                if q
            ],
        }
        ngraph.query(
            """
        MERGE (p:Parent {id: $parent_id})
        WITH p
        UNWIND $questions AS question
        CREATE (q:Question {id: question.id})
        SET q.text = question.text
        MERGE (q)<-[:HAS_QUESTION]-(p)
        WITH q, question
        CALL db.create.setVectorProperty(q, 'embedding', question.embedding)
        YIELD node
        RETURN count(*)
        """,
            params,
        )
        # Create vector index #CALL db.index.vector.createNodeIndex
        try:
            ngraph.query(
                "CALL db.index.vector.createNodeIndex('hypothetical_questions', "
                "'Question', 'embedding', 768, 'cosine')",
                {},
            )
        except ClientError:  # already exists
            pass
#CALL db.index.vector.createNodeIndex('hypothetical_questions', 'Question', 'embedding', 768, 'cosine')

In [76]:


# Ingest summaries

summary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            ("Generate a summary of the following input: {question}\n" "Summary:"),
        ),
    ]
)

summary_chain = summary_prompt | llm


In [77]:

for i, parent in enumerate(parent_documents):
    summary = summary_chain.invoke({"question": parent.page_content}).content
    params = {
        "parent_id": i,
        "summary": summary,
        "embedding": EMBEDDING_MODEL.embed_query(summary)# EMBEDDING_MODEL.get_embeddings([summary])[0].values,
    }
    ngraph.query(
        """
    MERGE (p:Parent {id: $parent_id})
    MERGE (p)-[:HAS_SUMMARY]->(s:Summary)
    SET s.text = $summary
    WITH s
    CALL db.create.setVectorProperty(s, 'embedding', $embedding)
    YIELD node
    RETURN count(*)
    """,
        params,
    )
    # Create vector index
    #CALL db.index.vector.createNodeIndex('typical_rag', 'Parent', 'embedding', 768, 'cosine')
    try:
        ngraph.query(
            "CALL db.index.vector.createNodeIndex('summary', "
            "'Summary', 'embedding', 768, 'cosine')",
            {},
        )
    except ClientError:  # already exists
        pass
# CALL db.index.vector.createNodeIndex('summary', 'Summary', 'embedding', 768, 'cosine')

In [78]:
# Typical RAG retriever
EMBEDDING_MODEL_NAME = 'textembedding-gecko@003'



typical_rag = Neo4jVector.from_existing_index(
    url = url,
    username = username,
    password = password,
    embedding = VertexAIEmbeddings(model_name=EMBEDDING_MODEL_NAME),
    index_name = 'typical_rag'
)


In [79]:

parent_query = """
MATCH (node)<-[:HAS_CHILD]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata LIMIT 1
"""

parent_vectorstore = Neo4jVector.from_existing_index(
    url = url,
    username = username,
    password = password,
    embedding = VertexAIEmbeddings(model_name=EMBEDDING_MODEL_NAME),
    index_name="parent_document",
    retrieval_query=parent_query,
)

In [80]:


# # Hypothetic questions retriever

hypothetic_question_query = """
MATCH (node)<-[:HAS_QUESTION]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata
"""

hypothetic_question_vectorstore = Neo4jVector.from_existing_index(
    url = url,
    username = username,
    password = password,
    embedding = VertexAIEmbeddings(model_name=EMBEDDING_MODEL_NAME),
    index_name="hypothetical_questions",
    retrieval_query=hypothetic_question_query,
)

In [81]:

# # # Summary retriever

summary_query = """
MATCH (node)<-[:HAS_SUMMARY]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata
"""

summary_vectorstore = Neo4jVector.from_existing_index(
    url = url,
    username = username,
    password = password,
    embedding = VertexAIEmbeddings(model_name=EMBEDDING_MODEL_NAME),
    index_name="summary",
    retrieval_query=summary_query,
)

In [82]:
retriever = typical_rag.as_retriever().configurable_alternatives(
    ConfigurableField(id="strategy"),
    default_key="typical_rag",
    parent_strategy=parent_vectorstore.as_retriever(),
    hypothetical_questions=hypothetic_question_vectorstore.as_retriever(),
    summary_strategy=summary_vectorstore.as_retriever(),
)

In [83]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [103]:
template = """Your are a financial analyst Answer the question based only on the following context:
{context}
Use this above information to answer queries about contract between organizations and give details about dates, persons etc
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [104]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(
    model="gemini-1.0-pro-vision",
    temperature=0,
    max_tokens=None,
    max_retries=6,
    stop=None,
    # other params...
)

In [105]:


chain = (
    RunnableParallel(
        {
            "context": itemgetter("question") | retriever | format_docs,
            "question": itemgetter("question"),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [106]:
# Add typing for input
class Question(BaseModel):
    question: str


chain = chain.with_types(input_type=Question)

In [107]:
original_query = "List the organizations involved?"
print(
    chain.invoke(
        {"question": original_query},
        {"configurable": {"strategy": "summary_strategy"}},
    )
)

* Province of Nova Scotia
* Halifax Regional Municipality


In [108]:
original_query = "Who have signed this document?"
print(
    chain.invoke(
        {"question": original_query},
        {"configurable": {"strategy": "parent_strategy"}},
    )
)

The document is signed by the Nova Scotia Minister of Finance, representing Her Majesty the Queen, and the Chief Administrative Officer of the Halifax Regional Municipality.


In [111]:
original_query = "What is subject?"
print(
    chain.invoke(
        {"question": original_query},
        {"configurable": {"strategy": "hypothetical_questions"}},
    )
)

Confidentiality of information shared during discussions about the Province providing SAP services to HRM.


In [110]:
original_query = "What is origin?"
print(
    chain.invoke(
        {"question": original_query},
        {"configurable": {"strategy": "typical_rag"}},
    )
)

June 25, 2013 Council authorization for the CAO to negotiate SAP support services with the Province.


In [92]:
original_query = "When is the meeting scheduled?"
print(
    chain.invoke(
        {"question": original_query},
        {"configurable": {"strategy": "parent_strategy"}},
    )
)

The meeting is scheduled for Tuesday, September 23, 2014 at 6:00 PM.


In [112]:
cypher_prompt = """
Generate a Cypher query to query a graph database based on the user's question {question}."""

In [113]:
original_query

'What is subject?'

In [147]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
allowed_nodes=["Person", "Country", "Organization", "Location", "Effective Date", "Signed Date", "heading"],
                                      allowed_relationships=["signed by", "Signed on", "Section"]
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
The question is:
{question}"""

In [148]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [149]:
from langchain.chains import GraphCypherQAChain
from langchain_google_genai import ChatGoogleGenerativeAI


In [150]:
chain = GraphCypherQAChain.from_llm(
    ChatVertexAI(model='gemini-1.5-pro', max_output_tokens=8192, temperature=0.0),
    graph=ngraph,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    verbose=True,
    validate_cypher=True
)

In [151]:
chain.invoke("get all the Organization")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n:Organization) RETURN n

Full Context:
[{'n': {'id': 'Mayor Savage and Members of Halifax Regional Council'}}, {'n': {'id': 'Halifax Regional Municipality'}}, {'n': {'id': 'Legal Services'}}, {'n': {'id': 'IBM'}}, {'n': {'id': 'Province of Nova Scotia'}}, {'n': {'id': 'Disclosing Party'}}, {'n': {'id': 'Receiving Party'}}, {'n': {'id': 'Halifax Regional Council'}}]

> Finished chain.


{'query': 'get all the Organization',
 'result': 'Mayor Savage and Members of Halifax Regional Council, Halifax Regional Municipality, Legal Services, IBM, Province of Nova Scotia, Disclosing Party, Receiving Party, and Halifax Regional Council are all organizations. \n'}

In [ ]:


chain = (
    RunnableParallel(
        {
            "context": itemgetter("question") | retriever | format_docs,
            "question": itemgetter("question"),
            
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)